### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{v_file_date}'")

In [0]:
countries_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
movies_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies").filter(f"file_date = '{v_file_date}'")

In [0]:
productions_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies").filter(f"file_date = '{v_file_date}'")

In [0]:
productions_countries_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries").filter(f"file_date = '{v_file_date}'")

#### Join entre "countries" y "productions_countries"

In [0]:
countries_prod_coun_df = countries_df.join(productions_countries_df, countries_df.country_id == productions_countries_df.country_id, 'inner').select(countries_df.country_name, productions_countries_df.movie_id,countries_df.country_id)


#### Join entre "productions_companies" y "movies_companies"

In [0]:
productions_companies_mov_comp_df = productions_companies_df.join(movies_companies_df,productions_companies_df.company_id == movies_companies_df.company_id, 'inner').select(productions_companies_df.company_name, movies_companies_df.movie_id,productions_companies_df.company_id)

#### Join "movies_df", "counries_prod_coun_df", "productions_companies_mov_comp_df"

- Filtras las peliculas donde su fecha de lanzamiento sea mayor o igual a 2000

In [0]:
movie_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
results_countries_prod_company_df = movie_filter_df.join(countries_prod_coun_df,movie_filter_df.movie_id == countries_prod_coun_df.movie_id,"inner").join(productions_companies_mov_comp_df,movie_filter_df.movie_id == productions_companies_mov_comp_df.movie_id,"inner")

- Agregar la columna "created_date"

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
results_df = results_countries_prod_company_df.select(movie_filter_df.movie_id,"country_id","company_id","title","budget","revenue","duration_time","release_date","country_name","company_name").withColumn("created_date", lit(v_file_date))

- Ordenar por la columna "release_date" de manera descendente

In [0]:
results_order_by_df = results_df.orderBy(results_df.title.asc())

#### Escribir los datos en el DataLake en formato "Delta" 

In [0]:
#ovewrite_partition(results_order_by_df,"movie_gold","results_country_prod_company","created_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date'
merge_delta_lake(results_order_by_df, 'movie_gold', 'results_country_prod_company', gold_folder_path, merge_condition, 'created_date')

In [0]:
%sql
SELECT *
FROM movie_gold.results_country_prod_company;